In [1]:
#Imports
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import os

In [2]:
# MNIST Dataset
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
#Hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

In [4]:
# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)«

In [5]:

x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [6]:
# 3 layered network

def build_three_layered_network():
    weights = {
        'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
        'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
        'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden_1])),
        'b2': tf.Variable(tf.random_normal([n_hidden_2])),
        'out': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
        # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
        # Output layer with linear activation
    pred = tf.matmul(layer_2, weights['out']) + biases['out']
    return pred


In [7]:
pred = build_three_layered_network()

In [8]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [9]:
init = tf.global_variables_initializer()

In [10]:
session = tf.Session()
session.run(init)

In [11]:
#initializing Saver 
saver = tf.train.Saver()
save_dir = 'checkpoints/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
save_path = os.path.join(save_dir, 'best_validation')

In [12]:
def optimize(training_epochs):
    for epoch in range(training_epochs):
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            session.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
    saver.save(session, save_path)
    print("Model saved in file: %s" % save_path)

In [13]:
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [14]:
def print_accuracy():
    acc = session.run(accuracy,feed_dict={x: mnist.test.images,y: mnist.test.labels})
    print("Accuracy on test-set: {0:.1%}".format(acc))

In [15]:
#Without any Optimization So accuracy will be very Low
print_accuracy()

Accuracy on test-set: 8.2%


In [16]:
# Optimize with 1 epoch
optimize(1)
print_accuracy()

Model saved in file: checkpoints/best_validation
Accuracy on test-set: 86.2%


In [17]:
#Closing The Session
session.close()

In [18]:
#initializing new Session and Load pretrained model
session = tf.Session()
saver = tf.train.import_meta_graph('checkpoints/best_validation.meta')
saver.restore(session,tf.train.latest_checkpoint('checkpoints/'))

In [19]:
#Print Accuricy which will be with
print_accuracy()

Accuracy on test-set: 86.2%


In [20]:
#Optimize restored graph for 3 more epochs
optimize(3)
print_accuracy()

Model saved in file: checkpoints/best_validation
Accuracy on test-set: 92.1%


In [21]:
#Closing Session
session.close()

In [22]:
#Loading last saved model
session = tf.Session()
saver = tf.train.import_meta_graph('checkpoints/best_validation.meta')
saver.restore(session,tf.train.latest_checkpoint('checkpoints/'))
print_accuracy()

Accuracy on test-set: 92.1%
